# <center> Vowpal Wabbit tutorial: blazingly fast learning
In this tutorial, we'll cover (both theoratically and in practice) two reasons of Vowpal Wabbit's exceptional training speed, namely, online learning and hashing trick. We'll try it out on the competition's data as well as with news, letters, movie reviews datasets and gigabytes of StackOverflow questions.

# Outline
1. Stochastic gradient descent and online learning
    - 1.1. SGD
    - 1.2. Online approach to learning
2. Categorical data processing: Label Encoding, One-Hot Encoding, Hashing trick
    - 2.1. Label Encoding
    - 2.2. One-Hot Encoding
    - 2.3. Hashing trick
3. Vowpal Wabbit
    - 3.1. News. Binary classification
    - 3.2. News. Multiclass classification
    - 3.3. IMDB reviews
    - 3.4. Classifying gigabytes of StackOverflow questions
4. VW and Spooky Author Identification 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.datasets import fetch_20newsgroups, load_files
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 1. Stochastic gradient descent and online learning
##  1.1. Stochastic gradient descent

Despite the fact that gradient descent is one of the first things learned in machine learning and optimization courses, it is hard to overrate one of it's modifications, namely, Stochastic Gradient Descent (SGD).

Lets recap that the very idea of gradient descent is to minimize some function by making small steps in the direction of fastest function decreasing. The method was named due to the following fact from calculus: vector $\nabla f = (\frac{\partial f}{\partial x_1}, \ldots \frac{\partial f}{\partial x_n})^T$ of partial derivatives of the function $f(x) = f(x_1, \ldots x_n)$ points to the direction of the fastest function growth. It means that by moving in the opposite direction (antigradient) it is possible to decrease the function value with the fastest rate.

<img src='https://habrastorage.org/files/4f2/75d/a46/4f275da467a44fc4a8d1a11007776ed2.jpg' width=70%>

Here is a snowboarder (me) in Sheregesh, Russian most popular winter resort. I highly recommended it if you like skiing or snowboarding. We place this picture not only for a good view but also for picturing the idea of gradient descent. If you have an aim to ride as fast as possible, you need to choose the way with steepest descent (as long as you stay alive). Calculating antigradient can be seen as evaluating the slope in each particular point.

**Example**

The paired regression problem can be solved with gradient descent. Let us predict one variable with another, say, height with weight and assume that these variables are lineary dependent. Here we are using the SOCR dataset. 

In [2]:
PATH_TO_ALL_DATA = '../Data/'


# 3. Vowpal Wabbit

[Vowpal Wabbit](https://github.com/JohnLangford/vowpal_wabbit) (VW) is one of the most wide-spread machine learning libraries in industry. It is prominent for high training speed and support of many training modes. Especially, we are interested in online learning with big and high-dimentional data. This is one of major merits of the library. Also, with hashing trick implemented, Vowpal Wabbit is a perfect choice for working with text data.

Shell is the main interface for VW. Well... I haven't found the way of installing VW in a Kaggle Kernel (hmm.. Kaggle, what about Docker?) so I've commented out the code in some cells in order not to spoil the output. 

In [3]:
!vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all 

Vowpal Wabbit reads data from files or from standard input stream (stdin) assuming the following format:

`[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features`

`Namespace=String[:Value]`

`Features=(String[:Value] )*`

here [] denotes non-mandatory elements, and (...)\* means some repeats. 

- **Label** is a number. In case of classification it is usually 1 and -1, in case of regression it is some real float value
- **Importance** is a number, it denotes sample weight during training. Setting this helps when working with imbalanced data.
- **Tag** is some string without spaces - it is a "name" of sample, VW saves it upon prediction. In order to separate Tag and Importance it is better to start Tag with the ' character.
- **Namespace** is for creation of separate feature spaces. 
- **Features** are object features inside **Namespace**. Features have weight 1.0 by default, but it can be changed, for example - feature:0.1. 


The following string matches the VW format:

```
1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park
```


Let's check it and run VW with this training sample:

In [4]:
! echo '1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park' | vw

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       10

finished run
number of examples per pass = 1
passes used = 1
weighted example sum = 1.000000
weighted label sum = 1.000000
average loss = 1.000000
best constant = 1.000000
best constant's loss = 0.000000
total feature number = 10


# 4. VW and Spooky Author Identification
And finally, we'll try to use Vowpal Wabbit in the task of identifiyng one of the three authors (Edgar Allan Poe, Mary Shelley, or HP Lovecraft) given pieces of their spooky texts.

Let's load the data.

In [5]:
train_texts = pd.read_csv('../Data/train.csv', index_col='id')
test_texts = pd.read_csv('../Data/test.csv', index_col='id')
sample_sub = pd.read_csv('../Data/sample_submission.csv', 
                         index_col='id')

Let's encode the authors.

In [6]:
author_code = {"EAP": 1, "MWS": 2,"HPL": 3}

In [7]:
train_texts["author_code"] = train_texts["author"].map(author_code)

This is going to be our simple validation scheme, we are just using the validation hold-out set.

In [8]:
train_texts_part, valid_texts = train_test_split(train_texts, test_size=0.01, random_state=17, 
                                                 stratify=train_texts["author_code"], shuffle=True)

In [9]:
train_texts_part.shape[0], valid_texts.shape[0]

(19383, 196)

To begin with, we are using only texts as features. Th following code will prepare the data to be fit into VW. 

In [10]:
def to_vw_only_text(out_vw, df, is_train=True):
    with open(out_vw, "w") as out:
        for i in range(df.shape[0]):
            
            if is_train:
                target = df["author_code"].iloc[i]
            else:
                # for the test set we can pick any target label – we don't need it actually
                target = 1 
                       
            # remove special VW symbols
            text = df["text"].iloc[i].strip().replace('|', '').replace(':', '').lower() 
            # leave only words of 3 and more chars
            words = re.findall("\w{3,}", text) 
            new_text = " ".join(words) 

            s = "{} |text {}\n".format(target, new_text)

            out.write(s)    

In [11]:
to_vw_only_text("../Data/vowpal_wabbit/train_part_only_text.vw", train_texts_part)

In [12]:
!head -2 $PATH_TO_ALL_DATA/vowpal_wabbit/train_part_only_text.vw

2 |text solitude only shall myself solitude shall thine
2 |text knowledge the worldly principles lord raymond would have ever prevented from applying him however deep distress might have been


In [13]:
to_vw_only_text("../Data/vowpal_wabbit/valid_only_text.vw", valid_texts)

In [14]:
!head -2 $PATH_TO_ALL_DATA/vowpal_wabbit/valid_only_text.vw

2 |text were interrupted attendant who announced that the staff raymond was assembled the council chamber
1 |text was not that feared look upon things horrible but that grew aghast lest there should nothing see


In [15]:
to_vw_only_text("../Data/vowpal_wabbit/train_only_text.vw", train_texts)

In [16]:
!head -2 $PATH_TO_ALL_DATA/vowpal_wabbit/train_only_text.vw

1 |text this process however afforded means ascertaining the dimensions dungeon might make its circuit and return the point whence set out without being aware the fact perfectly uniform seemed the wall
3 |text never once occurred that the fumbling might mere mistake


In [17]:
to_vw_only_text("../Data/vowpal_wabbit/test_only_text.vw", test_texts, is_train=False)

In [18]:
!head -2 $PATH_TO_ALL_DATA/vowpal_wabbit/test_only_text.vw

1 |text still urged our leaving ireland with such inquietude and impatience father thought best yield
1 |text fire wanted fanning could readily fanned with newspaper and the government grew weaker have doubt that leather and iron acquired durability proportion for very short time there was not pair bellows all rotterdam that ever stood need stitch required the assistance hammer


Here we train a VW model (actuall 3 one-against-all models), we use 28 bits for feature hashing resulting in $2^{28} \approx 2.7 \times 10^8$ features. The loss is set to logistic as it's works well for classification (and it's also our evaluation metric in the competition). We incorporate bigrams and perform 10 passes over the whole dataset.

In [19]:
!vw --oaa 3 ../Data/vowpal_wabbit/train_part_only_text.vw -f ../Data/vowpal_wabbit/model_only_text_part.vw -b 28 --random_seed 17 \
--loss_function logistic --ngram 2 --passes 10 -k -c

Generating 2-grams for all namespaces.
final_regressor = ../Data/vowpal_wabbit/model_only_text_part.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../Data/vowpal_wabbit/train_part_only_text.vw.cache
Reading datafile = ../Data/vowpal_wabbit/train_part_only_text.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        2        1       14
0.500000 0.000000            2            2.0        2        2       38
0.500000 0.500000            4            4.0        3        2       54
0.375000 0.250000            8            8.0        3        3       80
0.500000 0.625000           16           16.0        1        3       56
0.531250 0.562500           32           32.0        2        3       74
0.593750 0.656250           64           64.0        1        2  

In [20]:
%%time
!vw -i ../Data/vowpal_wabbit/model_only_text_part.vw -t -d ../Data/vowpal_wabbit/valid_only_text.vw -p ../Data/vowpal_wabbit/valid_pred1.txt --random_seed 17 -r ../Data/vowpal_wabbit/valid_prob1.txt

Generating 2-grams for all namespaces.
only testing
predictions = ../Data/vowpal_wabbit/valid_pred1.txt
raw predictions = ../Data/vowpal_wabbit/valid_prob1.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../Data/vowpal_wabbit/valid_only_text.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2       28
0.000000 0.000000            2            2.0        1        1       34
0.000000 0.000000            4            4.0        3        3       46
0.000000 0.000000            8            8.0        1        1       32
0.062500 0.125000           16           16.0        2        2       56
0.062500 0.062500           32           32.0        2        2       10
0.125000 0.187500           64           64.0        2        2       48
0.132812 0.140625      

We get classification scores for each validation sample, so we'll perform sigmoid transformation to map them into [0,1] range. Further, we calculate the logistic loss between target vector in validation data set and the transformed predictions. It's handy to write a special function for all these steps.

In [21]:
def evaluate_vw_prediction(path_to_vw_pred_probs, is_test=False, target=None, write_submission=False,
                          submission_file=None, test_index=test_texts.index, columns=['EAP', 'MWS', 'HPL']):
    def sigmoid(z):
        return 1 / (1 + np.exp(-z)) 
    
    with open(path_to_vw_pred_probs) as pred_file:
        pred_probs =  np.array([[float(pair.split(':')[1]) for pair in line.strip().split()] 
                         for line in pred_file.readlines()])
        pred_probs  = sigmoid(pred_probs)
        
        if target is not None and not is_test:
            print(log_loss(target, pred_probs))
        
        if write_submission and submission_file is not None:
            subm_df = pd.DataFrame(pred_probs, columns=columns)
            subm_df.index = test_index
            subm_df.to_csv(submission_file)

In [22]:
evaluate_vw_prediction(os.path.join(PATH_TO_ALL_DATA, 'vowpal_wabbit/valid_prob1.txt'), 
                       target=valid_texts['author_code'])

0.473879356865


Now it's high time to train VW on the full training set, make predictions for the test set and submit them to Kaggle.

In [23]:
!vw --oaa 3 ../Data/vowpal_wabbit/train_only_text.vw -f ../Data/vowpal_wabbit/model_only_text.vw -b 28 --random_seed 17 \
--loss_function logistic --ngram 2 --passes 10 -k -c --quiet

In [24]:
%%time
!vw -i ../Data/vowpal_wabbit/model_only_text.vw -t -d ../Data/vowpal_wabbit/test_only_text.vw -p ../Data/vowpal_wabbit/test_pred1.txt --random_seed 17 -r ../Data/vowpal_wabbit/test_prob1.txt --quiet

CPU times: user 9.68 ms, sys: 9.57 ms, total: 19.2 ms
Wall time: 666 ms


In [25]:
evaluate_vw_prediction(os.path.join(PATH_TO_ALL_DATA, 'vowpal_wabbit/test_prob1.txt'), 
                       is_test=True, write_submission=True,
                       submission_file='../predictions/submission1_only_text.csv')

In [26]:
!head -3 ../predictions/submission1_only_text.csv

id,EAP,MWS,HPL
id02310,0.21302281257134767,0.7462194862375665,0.02739253612496469
id24541,0.8236463649690904,0.0062378023959290705,0.143383189058737


With this submission we get 0.43187 on the [Public Leaderboard](https://www.kaggle.com/c/spooky-author-identification/leaderboard).

Let's add some features

In [27]:
max_words_in_text = train_texts['text'].apply(lambda text: len(re.findall("\w{3,}", text.strip()))).max()
max_unique_words_in_text = train_texts['text'].apply(lambda text: len(set(re.findall("\w{3,}", text.strip())))).max()
max_aver_word_len_in_text = train_texts['text'].apply(lambda text: 
                                                      sum([len(w) for w in re.findall("\w{3,}", text.strip())]) / 
                                                      len(re.findall("\w{3,}", text.strip()))).max()

In [28]:
max_words_in_text, max_unique_words_in_text, max_aver_word_len_in_text

(667, 400, 11.0)

In [29]:
def to_vw_text_and_some_features(out_vw, df, is_train=True):
    with open(out_vw, "w") as out:
        for i in range(df.shape[0]):
            
            if is_train:
                target = df["author_code"].iloc[i]
            else:
                # for the test set we can pick any target label – we don't need it actually
                target = 1 
                       
            # remove special VW symbols
            text = df["text"].iloc[i].strip().replace('|', '').replace(':', '').lower() 
            # leave only words of 3 and more chars
            words = re.findall("\w{3,}", text) 
            new_text = " ".join(words)    
            
            num_words = round(len(words) / max_words_in_text, 4)
            num_uniq_words = round(len(set(words)) / max_unique_words_in_text, 4)
            aver_word_len = round(sum([len(w) for w in words]) / len(words) / max_aver_word_len_in_text, 4)

            features = [num_words, num_uniq_words, aver_word_len] 
            features_vw = ' '.join(['{}:{}'.format(i[0], i[1]) for i in zip(range(len(features)), features)])
            s = "{} |text {} |num {}\n".format(target, new_text, features_vw)

            out.write(s)   
 

In [30]:
to_vw_text_and_some_features("../Data/vowpal_wabbit/train_part_text_feat.vw", train_texts_part)

In [31]:
!head -2 $PATH_TO_ALL_DATA/train_part_text_feat.vw

head: ../Data//train_part_text_feat.vw: No such file or directory


In [32]:
to_vw_text_and_some_features("../Data/vowpal_wabbit/valid_text_feat.vw", valid_texts)

In [33]:
to_vw_text_and_some_features("../Data/vowpal_wabbit/train_text_feat.vw", train_texts)

In [34]:
to_vw_text_and_some_features("../Data/vowpal_wabbit/test_text_feat.vw", test_texts, is_train=False)

In [35]:
!vw --oaa 3 ../Data/vowpal_wabbit/train_part_text_feat.vw -f ../Data/vowpal_wabbit/model_text_feat_part.vw -b 28 --random_seed 17 \
--loss_function logistic --ngram 2 --passes 10 -k -c --quiet

In [36]:
%%time
!vw -i ../Data/vowpal_wabbit/model_text_feat_part.vw -t -d ../Data/vowpal_wabbit/valid_text_feat.vw -p ../Data/vowpal_wabbit/valid_pred2.txt --random_seed 17 -r ../Data/vowpal_wabbit/valid_prob2.txt --quiet

CPU times: user 5.24 ms, sys: 9.11 ms, total: 14.3 ms
Wall time: 376 ms


In [37]:
evaluate_vw_prediction(os.path.join(PATH_TO_ALL_DATA, 'vowpal_wabbit/valid_prob2.txt'), 
                       target=valid_texts['author_code'])

0.438015789069


In [38]:
!vw --oaa 3 ../Data/vowpal_wabbit/train_text_feat.vw -f ../Data/vowpal_wabbit/model_text_feat.vw -b 28 --random_seed 17 \
--loss_function logistic --ngram 2 --passes 10 -k -c --quiet

In [39]:
%%time
!vw -i ../Data/vowpal_wabbit/model_text_feat.vw -t -d ../Data/vowpal_wabbit/test_text_feat.vw -p ../Data/vowpal_wabbit/test_pred2.txt --random_seed 17 -r ../Data/vowpal_wabbit/test_prob2.txt --quiet

CPU times: user 11 ms, sys: 9.15 ms, total: 20.1 ms
Wall time: 657 ms


In [40]:
evaluate_vw_prediction(os.path.join(PATH_TO_ALL_DATA, 'vowpal_wabbit/test_prob2.txt'), 
                       is_test=True, write_submission=True,
                       submission_file='../predictions/submission2_text_feat.csv')

With this we get 0.43267 on public LB so it doesn't seem like a major improvement. However,  we'll take into account that our holdout score estimate is calculated with 5874 samples while the public leaderboard one – with $\approx$ 2517 samples (30% $\times$ 8392). Finally, we'll calculate the weighted sum of CV and LB scores to evaluate our submissions.

In [41]:
def validate_submission_local_and_lb_mix(local_score, public_lb_score, local_size=5874, public_lb_size=2517):
    return 1. / (local_size + public_lb_size) * (local_size * local_score +
                                                public_lb_size * public_lb_score)

In [42]:
# first submission
validate_submission_local_and_lb_mix(local_score=.47951, public_lb_score=.43187)

0.4652197032534859

In [43]:
# second submission
validate_submission_local_and_lb_mix(local_score=.469, 
                                      public_lb_score=.43267)

0.45810229889166965

It seems 3 features helped here to lower logloss. However, feature engineering is not the main goal of this tutorial, there are already dozens of nice ones in [Kernels](https://www.kaggle.com/c/spooky-author-identification/kernels). 

You can experiment with lots of other features, bunches of features and techniques (word2vec, LDA, topic modelling, to name just a few possible approaches). Luckily, Vowpal Wabbit serves ideally for performing lots of "design-implement-check" iterations. Try it out and you'll definitely gain a new helpful skill!

## Useful links
- Official VW [documentation](https://github.com/JohnLangford/vowpal_wabbit/wiki) on Github
- [Chapter](http://www.deeplearningbook.org/contents/numerical.html) "Numeric Computation" of the [Deep Learning book](http://www.deeplearningbook.org/)
- "Command-line Tools can be 235x Faster than your Hadoop Cluster", [post](https://aadrake.com/command-line-tools-can-be-235x-faster-than-your-hadoop-cluster.html)
- Benchmarking various ML algorithms on Criteo 1TB dataset, [GitHub](https://github.com/rambler-digital-solutions/criteo-1tb-benchmark)
- [FastML.com](http://fastml.com/blog/categories/vw/), category VW

<img src="https://habrastorage.org/webt/_r/lz/wb/_rlzwbzedhlivdnhvfzk1apnzss.jpeg" width=50%>